In [54]:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import numpy as np

# DiCE imports
import dice_ml
from dice_ml.utils import helpers  

In [72]:
dataset = helpers.load_adult_income_dataset()
display(dataset.head())

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,28,Private,Bachelors,Single,White-Collar,White,Female,60,0
1,30,Self-Employed,Assoc,Married,Professional,White,Male,65,1
2,32,Private,Some-college,Married,White-Collar,White,Male,50,0
3,20,Private,Some-college,Single,Service,White,Female,35,0
4,41,Self-Employed,Some-college,Married,White-Collar,White,Male,50,0


In [56]:
# description of transformed features
adult_info = helpers.get_adult_data_info()
adult_info

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

In [57]:
#dataset.drop(['age','race','gender'], axis=1, inplace=True)

In [58]:
dataset['income'].value_counts()

0    19820
1     6228
Name: income, dtype: int64

Data is fairly imbalanced which might cause the ML model to overfit by predicting the most common labels (essentially obtaining random prediction power AUC-ROC score of 0.5) Furthermore, we split the dataset into train and test sets.

In [59]:
from collections import Counter

target = dataset["income"]
X = dataset.drop(['income'],axis=1)


ros = RandomUnderSampler()
# resampling X, y
dataset, target = ros.fit_resample(X, target)
# new class distribution 
print(Counter(target))

dataset = pd.concat([dataset,target],axis=1)

train_dataset, test_dataset, y_train, y_test = train_test_split(dataset,
                                                                target,
                                                                test_size=0.2,
                                                                random_state=0,
                                                                stratify=target)
x_train = train_dataset.drop('income', axis=1)
x_test = test_dataset.drop('income', axis=1)

Counter({0: 6228, 1: 6228})


In [60]:
# Step 1: dice_ml.Data
d = dice_ml.Data(dataframe=train_dataset, continuous_features=['age','hours_per_week'], outcome_name='income')

### Loading the ML model

In [70]:
numerical = ["hours_per_week"]
categorical = x_train.columns.difference(numerical)

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.

clf_rf = Pipeline(steps=[('preprocessor', transformations),
                      ('clf', RandomForestClassifier(random_state=42))])

clf_svm = Pipeline(steps=[('preprocessor', transformations),
                      ('clf', SVC(random_state=42))])

#param_range = [1, 200]

grid_params_rf = [{'clf__criterion': ['gini', 'entropy'],
                   'clf__max_features': ['auto', 'sqrt'],
                   'clf__n_estimators': [int(x) for x in np.linspace(start = 128, stop = 384, num = 32)],
                   'clf__min_samples_split': [2, 5, 10],
                   'clf__max_depth': [int(x) for x in np.linspace(start = 5, stop = 8, num = 1)]}]

grid_params_svm = [{'clf__kernel': ['poly', 'rbf','sigmoid'], 
        'clf__C': [0.1, 1, 10, 100, 1000]}]
        #'clf__gamma': [1, 0.1, 0.01, 0.001, 0.0001]}]

jobs = -1

RF = GridSearchCV(estimator=clf_rf,
            param_grid=grid_params_rf,
            scoring='roc_auc',
            cv=10, 
            n_jobs=jobs)


SVM = GridSearchCV(estimator=clf_svm,
            param_grid=grid_params_svm,
            scoring='roc_auc',
            cv=10)

grids = [SVM]

In [71]:
# Creating a dict for our reference
grid_dict = {#0: 'Random Forest', 
        0: 'Support Vector Machine'}

best_rf = {'score': 0, 'best_model':0}
best_svm = {'score': 0, 'best_model':0}

# Fit the grid search objects
print('Performing model optimizations...')

for idx, model in enumerate(grids):
    
    print('\nEstimator: %s' % grid_dict[idx])
    model.fit(x_train, y_train)
    print('Best params are : %s' % model.best_params_)
    
    # Best training data accuracy
    print('Best training accuracy: %.3f' % model.best_score_)
    
    # Predict on test data with best params
    y_pred = model.predict(x_test)
    print('Test set accuracy score for best params: %.3f ' % accuracy_score(y_test, y_pred))
    print('Precision score: {}'.format(precision_score(y_test, y_pred)))
    print('Recall score: {}'.format(recall_score(y_test, y_pred)))
    print('F1 score: {}'.format(f1_score(y_test, y_pred)))
    print('AUC-ROC score: {}'.format(roc_auc_score(y_test, y_pred)))
    
    # Track best (highest test accuracy) model
#     if idx == 0:
#         if roc_auc_score(y_test, y_pred) > best_rf['score']:
#             best_rf['score'] = roc_auc_score(y_test, y_pred)
#             best_rf['best_model'] = model.best_params_ 
    
    if idx == 0:
        if roc_auc_score(y_test, y_pred) > best_svm['score']:
            best_svm['score'] = roc_auc_score(y_test, y_pred)
            best_svm['best_model'] = model.best_params_
            
# save dict to file
import json

#with open('best_adultincome_rf_params.txt', 'w') as file:
     #file.write(json.dumps(best_rf)) # use `json.loads` to do the reverse
        
with open('best_adultincome_svm_params.txt', 'w') as file:
     file.write(json.dumps(best_svm)) # use `json.loads` to do the reverse


Performing model optimizations...

Estimator: Support Vector Machine
Best params are : {'clf__C': 1, 'clf__kernel': 'rbf'}
Best training accuracy: 0.862
Test set accuracy score for best params: 0.778 
Precision score: 0.7474964234620887
Recall score: 0.8386837881219904
F1 score: 0.7904689863842662
AUC-ROC score: 0.7776886035313002


In [ ]:
rf_model = RandomForestClassifier()
svm_model = SVC()
rf_params = best_rf['best_model']
svm_params = best_svm['best_model']
print(svm_params)
print(rf_params)
#rf_model.set_params(**rf_params)
# svm_model.set_params(**svm_params)

In [ ]:
clf_rf = Pipeline(steps=[('preprocessor', transformations),
                      ('clf', rf_model)])

clf_svm = Pipeline(steps=[('preprocessor', transformations),
                      ('clf', svm_model)])

clf_rf.fit(x_train, y_train)

## Generating counterfactual examples using DiCE

We now initialize the DiCE explainer, which needs a dataset and a model. DiCE provides local explanation for the model *m* and requires an query input whose outcome needs to be explained. 

In [41]:
# Using sklearn backend
m = dice_ml.Model(model=model, backend="sklearn")
# Using method=random for generating CFs
exp = dice_ml.Dice(d, m, method="random")

In [42]:
e1 = exp.generate_counterfactuals(x_test[0:1], total_CFs=2, desired_class="opposite")
e1.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:00<00:00,  5.92it/s]

Query instance (original outcome : 0)


,workclass,education,marital_status,occupation,hours_per_week,income
0,Other/Unknown,HS-grad,Married,Other/Unknown,40,0



Diverse Counterfactual set (new outcome: 1.0)


,workclass,education,marital_status,occupation,hours_per_week,income
0,-,Doctorate,Divorced,-,-,1
1,Self-Employed,-,-,-,18.0,1


The `show_only_changes` parameter highlights the changes from the query instance. If you would like to see the full feature values for the counterfactuals, set it to False.

In [40]:
e1.visualize_as_dataframe(show_only_changes=False)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,HS-grad,Married,Blue-Collar,White,Female,38,0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,Prof-school,Married,Blue-Collar,White,Female,94,1
1,29,Private,Assoc,Married,Service,White,Female,38,1


That's it! You can try generating counterfactual explanations for other examples using the same code. 
It is also possible to restrict the features to vary while generating the counterfactuals, and to specify permitted range of features within which the counterfactual should be generated. 

### Local feature importance scores

These scores are computed for a given query instance (input point) by summarizing a set of counterfactual examples around the point. 

In [43]:
query_instance = x_test[0:1]
imp = exp.local_feature_importance(query_instance, total_CFs=10)
print(imp.local_importance)

100%|██████████| 1/1 [00:00<00:00,  3.09it/s]

[{'education': 0.8, 'workclass': 0.4, 'occupation': 0.3, 'age': 0.2, 'marital_status': 0.1, 'gender': 0.1, 'race': 0.0, 'hours_per_week': 0.0}]


### Global feature importance scores

A global importance score per feature can be estimated by aggregating the scores over individual inputs. The more the inputs, the better the estimate for global importance of a feature.

In [44]:
query_instances = x_test[0:20]
imp = exp.global_feature_importance(query_instances)
print(imp.summary_importance)

100%|██████████| 20/20 [00:06<00:00,  3.06it/s]

{'education': 0.7, 'occupation': 0.32, 'marital_status': 0.255, 'age': 0.23, 'hours_per_week': 0.19, 'race': 0.14, 'workclass': 0.13, 'gender': 0.1}
